<a href="https://colab.research.google.com/github/Lukithas/Dashboard_ODS13/blob/main/Mini_Projeto_Web_com_visualiza%C3%A7%C3%A3o__dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.8 MB/s eta 0:00:00


In [2]:
# CÉLULA 1: PREPARAÇÃO DO AMBIENTE


!pip install -q streamlit pyngrok

# 1. Importa a biblioteca do Google Drive e monta a pasta
from google.colab import drive
print("Por favor, autorize o acesso à sua conta Google Drive.")
drive.mount('/content/drive')
print("✅ Google Drive montado com sucesso em /content/drive")

Por favor, autorize o acesso à sua conta Google Drive.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive montado com sucesso em /content/drive


In [3]:
# CÉLULA 2: CRIAÇÃO DO ARQUIVO DA APLICAÇÃO

%%writefile app.py

# --- Início do código da aplicação Streamlit ---
import streamlit as st
import pandas as pd
import numpy as np

# --- CONFIGURAÇÃO DA PÁGINA ---
st.set_page_config(
    page_title="EDA ODS 13 - Emissões no Brasil",
    page_icon="🌍",
    layout="wide"
)

# --- FUNÇÃO DE CARREGAMENTO E CACHE DOS DADOS ---
@st.cache_data
def carregar_dados(caminho_arquivo, nome_aba):
    try:
        df_wide = pd.read_excel(caminho_arquivo, sheet_name=nome_aba)
        colunas_id = [
            'Emissão/Remoção/Bunker', 'Gás', 'Setor de emissão', 'Categoria emissora',
            'Sub-categoria emissora', 'Produto ou sistema', 'Detalhamento', 'Recorte',
            'Atividade geral', 'Estado', 'Bioma'
        ]
        colunas_valor = [col for col in df_wide.columns if isinstance(col, int)]
        df_long = pd.melt(
            df_wide, id_vars=colunas_id, value_vars=colunas_valor,
            var_name='Ano', value_name='Emissao_Tons'
        )
        df_long['Emissao_Tons'].fillna(0, inplace=True)
        df_long['Ano'] = pd.to_numeric(df_long['Ano'])
        df_emissoes = df_long[df_long['Emissão/Remoção/Bunker'] == 'Emissão'].copy()
        return df_emissoes
    except FileNotFoundError:
        st.error(f"ERRO: Arquivo não encontrado em '{caminho_arquivo}'. O Google Drive está montado corretamente?")
        return None
    except KeyError:
        st.error(f"ERRO: A aba '{nome_aba}' não foi encontrada. Verifique o nome.")
        return None
    except Exception as e:
        st.error(f"Ocorreu um erro inesperado: {e}")
        return None

# --- INTERFACE PRINCIPAL DA APLICAÇÃO ---
st.title("🌍 Análise Exploratória de Emissões de GEE no Brasil")
st.markdown("Dashboard interativo para explorar dados da ODS 13 no contexto da COP30.")

# O script agora simplesmente assume que o caminho existe
file_path = '/content/drive/My Drive/Programacao_WEB/Dados-nacionais-v12.xlsx'
nome_da_aba = 'Dados'
df_emissoes = carregar_dados(file_path, nome_da_aba)

if df_emissoes is not None:
    st.sidebar.header("Filtros Interativos")
    ano_inicio, ano_fim = st.sidebar.slider("Selecione o Período:", min_value=int(df_emissoes['Ano'].min()), max_value=int(df_emissoes['Ano'].max()), value=(int(df_emissoes['Ano'].min()), int(df_emissoes['Ano'].max())))
    setores_disponiveis = df_emissoes['Setor de emissão'].unique()
    setor_selecionado = st.sidebar.selectbox("Selecione um Setor:", options=['Todos os Setores'] + list(setores_disponiveis))
    estados_disponiveis = sorted(df_emissoes[~df_emissoes['Estado'].isin(['Não se aplica', 'BR'])]['Estado'].unique())
    estado_selecionado = st.sidebar.selectbox("Selecione um Estado:", options=['Brasil (Total)'] + estados_disponiveis)

    dados_filtrados = df_emissoes[(df_emissoes['Ano'] >= ano_inicio) & (df_emissoes['Ano'] <= ano_fim)]

    titulo_analise = f"Análise para: {estado_selecionado}"
    if setor_selecionado != 'Todos os Setores':
        dados_filtrados = dados_filtrados[dados_filtrados['Setor de emissão'] == setor_selecionado]
        titulo_analise += f" | Setor: {setor_selecionado}"
    else:
        titulo_analise += " | Todos os Setores"
    if estado_selecionado != 'Brasil (Total)':
        dados_filtrados = dados_filtrados[dados_filtrados['Estado'] == estado_selecionado]

    st.header(titulo_analise)
    st.subheader("Evolução das Emissões no Período Selecionado")
    emissoes_por_ano = dados_filtrados.groupby('Ano')['Emissao_Tons'].sum()
    st.line_chart(emissoes_por_ano)

    if setor_selecionado == 'Todos os Setores':
        st.subheader("Contribuição dos Setores para as Emissões Totais")
        emissoes_setor_ano = dados_filtrados.groupby(['Ano', 'Setor de emissão'])['Emissao_Tons'].sum().unstack(fill_value=0)
        st.area_chart(emissoes_setor_ano)

    if st.checkbox("Mostrar dados brutos filtrados"):
        st.dataframe(dados_filtrados)
else:
    st.warning("A aplicação não pode continuar porque os dados não foram carregados. Verifique os erros acima.")

Writing app.py


In [5]:
# CÉLULA 3: EXECUÇÃO DA APLICAÇÃO

from pyngrok import ngrok
import time
import subprocess


AUTHTOKEN = "33yXJYjXpVQYgBUYGKBUT5tM0xP_srbwA2u1X2apw44cXLZb"
ngrok.set_auth_token(AUTHTOKEN)

# Mata qualquer processo ngrok ou streamlit que possa estar rodando
ngrok.kill()
# O comando pkill força o encerramento de qualquer processo streamlit anterior
!pkill streamlit

# Inicia o streamlit em background
print("🚀 Iniciando o servidor Streamlit em segundo plano...")
process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# Aumentamos o tempo de espera para garantir que o servidor esteja totalmente pronto
print("⏳ Aguardando 15 segundos para a inicialização completa do servidor...")
time.sleep(15)

# Verifica se o processo do Streamlit ainda está em execução
if process.poll() is None:
    print("✅ Servidor Streamlit iniciado com sucesso.")
    # Tenta criar o túnel e imprimir a URL
    print("🔗 Criando o túnel com o ngrok...")
    try:
        public_url = ngrok.connect(8501)
        print("------------------------------------------------------------------")
        print(f"🎉 Aplicação pronta! Acesse pelo link: {public_url}")
        print("------------------------------------------------------------------")
    except Exception as e:
        print(f"❌ Falha ao criar o túnel do ngrok: {e}")
else:
    print("❌ ERRO: O servidor Streamlit falhou ao iniciar.")
    print("Isso geralmente acontece por um erro no arquivo 'app.py' ou problema de permissão no Google Drive.")
    print("Verifique se você autorizou o acesso ao Drive e se o caminho/nome da aba no arquivo Excel está correto.")

🚀 Iniciando o servidor Streamlit em segundo plano...
⏳ Aguardando 15 segundos para a inicialização completa do servidor...
✅ Servidor Streamlit iniciado com sucesso.
🔗 Criando o túnel com o ngrok...
------------------------------------------------------------------
🎉 Aplicação pronta! Acesse pelo link: NgrokTunnel: "https://reynaldo-preambitious-tierra.ngrok-free.dev" -> "http://localhost:8501"
------------------------------------------------------------------


In [ ]:
print(dados_emissoes.columns.tolist())

['Emissão/Remoção/Bunker', 'Gás', 'Setor de emissão', 'Categoria emissora', 'Sub-categoria emissora', 'Produto ou sistema', 'Detalhamento', 'Recorte', 'Atividade geral', 'Estado', 'Bioma', 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
